In [1]:
import json
import time
import requests
import hmac
import hashlib
import time
import pandas as pd
import cbpro
from datetime import datetime, timedelta
from joblib import load
from urllib3.exceptions import ProtocolError, ConnectionError

In [2]:
with open('../coinbase-secrets.json', 'r') as jfile:
    secrets = json.loads(jfile.read())
    jfile.close()

## Coinbase Pro

In [3]:
public_client = cbpro.PublicClient()

In [4]:
public_client.get_product_ticker(product_id='ETH-USD')

{'trade_id': 134998649,
 'price': '2042.13',
 'size': '15.45',
 'time': '2021-07-02T12:09:54.676668Z',
 'bid': '2042.12',
 'ask': '2042.13',
 'volume': '198522.82638557'}

In [5]:
time_data = public_client.get_time()
end = '2021-06-28T23:59:59.999Z'
start = '2021-06-28T23:00:00.000Z'

In [6]:
columns = ['timestamp', 'low', 'high', 'open', 'close', 'volume']

In [7]:
historic_data = public_client.get_product_historic_rates(
    product_id = 'ETH-USD', 
    start=str(start), 
    end=str(end), 
    granularity=3600
)
historic_data = pd.DataFrame(historic_data, columns=columns)
historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp'], unit='s')
historic_data

,timestamp,low,high,open,close,volume
0,2021-06-28 23:00:00,2062.35,2097.5,2097.5,2083.79,9784.226516


In [8]:
end = time_data['iso']
print(end)

2021-07-02T12:09:57.006Z


In [9]:
end_datetime = datetime.fromtimestamp(time_data['epoch'])
start_datetime = end_datetime - timedelta(hours=1)
end_iso = end_datetime.isoformat()
start_iso = start_datetime.isoformat()

In [10]:
historic_data = public_client.get_product_historic_rates(
    product_id = 'ETH-USD', 
    start=start_iso, 
    end=end_iso, 
    granularity=3600
)
historic_data = pd.DataFrame(historic_data, columns=columns)
historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp'], unit='s')
historic_data

,timestamp,low,high,open,close,volume
0,2021-07-02 08:00:00,2052.54,2079.6,2061.88,2060.21,5549.275835


In [11]:
model = load('../models/linear_model.joblib')
batch = historic_data[['open', 'high', 'low', 'close', 'volume']].values
prediction = model.predict(batch)
prediction

array([2059.55241498])

In [30]:
class AssetTrader(object):
    
    def __init__(self, asset: str, api_secret: str, api_key: str, passphrase: str, use_sandbox: bool = True):
        self.asset = asset
        self.api_secret = api_secret
        self.public_client = cbpro.PublicClient()
        api_url = ''
        if use_sandbox:
            api_url = "https://api-public.sandbox.pro.coinbase.com"
        else:
            api_url = "https://api.pro.coinbase.com"
        
        self.private_client = cbpro.AuthenticatedClient(
            key=api_key, 
            b64secret=api_secret.encode(), 
            passphrase=passphrase,
            api_url=api_url
        )
        self.accounts = self.private_client.get_accounts()
        for account in self.accounts:
            if account['currency'] == 'USD':
                self.usd_wallet = account['id']
            elif account['currency'] == self.asset.split('-')[0]:
                self.asset_wallet = account['id']
        
        
    def _get_start_end_iso_times(self, hours: int = 1):
        """
        From the current iso formatted timestamp, this generates
        a start and end datetime that are 1 hour apart.
        
        :returns: (tuple) Contains (start, end) datetimes.
        """
        
        time_data = self.public_client.get_time()
        
        end_datetime = datetime.fromtimestamp(time_data['epoch'])
        start_datetime = end_datetime - timedelta(hours=hours)
        
        end_iso = end_datetime.isoformat()
        start_iso = start_datetime.isoformat()
        return (start_iso, end_iso)

        
    def get_asset_details_last_hour(self, start: str, end: str, granularity: int = 3600):
        """
        Retrieves hourly open, high, low, close, and volume for the given asset
        over the time range on start to end broken into granularity seconds.
        
        :param start: (str) ISO-8601 formatted timestamp.
        :param end: (str) ISO-8601 formatted timestamp.
        :param granularity: (int) Number of seconds per interval between start and end.
        :returns: (np.array) Array containing the detailed asset price data.
        """
        try:
            historic_data = public_client.get_product_historic_rates(
                product_id = self.asset, 
                start=start, 
                end=end, 
                granularity=granularity
            )
        except (ProtocolError, ConnectionError):
            time.sleep(5)
            historic_data = public_client.get_product_historic_rates(
                product_id = self.asset, 
                start=start, 
                end=end, 
                granularity=granularity
            )
        historic_data = pd.DataFrame(historic_data, columns=columns)
        historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp'], unit='s')
        historic_data = historic_data[['open', 'high', 'low', 'close', 'volume']]
        
        return historic_data.values
    
    def get_account_balance(self, account_id: str):
        """Retrieves the account balance for a given account_id"""
        
        account_details = self.private_client.get_account(account_id)
        return float(account_details['balance'])
    
    def place_order(self, account_id: str, amount: float, order_type: str = 'buy'):
        """
        Given an account and amount, checks to see if we have 
        the required funds and places the order if possible.
        
        """
        order_type = order_type.lower()
        assert order_type in ['buy', 'sell'], f'Invalid order_type passed, not in [buy, sell]: {order_type}'
        
        usd_balance = self.get_account_balance(self.usd_wallet)
        asset_balance = self.get_account_balance(self.asset_wallet)
        
        if amount > usd_balance:
            amount = usd_balance
            
        response = self.private_client.place_market_order(
            product_id=self.asset, 
            side=order_type, 
            funds=str(amount)
        )
        return response

In [32]:
asset_trader = AssetTrader(
    asset='BTC-USD', 
    api_secret=secrets['SECRET_SANDBOX'], 
    api_key=secrets['KEY_SANDBOX'], 
    passphrase=secrets['PASSPHRASE_SANDBOX'],
    use_sandbox=True
)

In [14]:
start, end = asset_trader._get_start_end_iso_times()
last_hour = asset_trader.get_asset_details_last_hour(start, end)
last_hour

array([[33364.6      , 33600.       , 33238.4      , 33373.24     ,
          376.2505076]])

In [28]:
asset_trader.get_account_balance(asset_trader.usd_wallet)

89.8670775916525

In [29]:
asset_trader.get_account_balance(asset_trader.asset_wallet)

0.00030121

In [17]:
response = asset_trader.place_order(
    account_id=asset_trader.usd_wallet, 
    amount=10.0,
    order_type='buy'
)

In [27]:
orders = asset_trader.private_client.get_order(response['id'])
orders

{'id': 'a89c6a4d-3565-4254-8472-217fe4c82b77',
 'product_id': 'BTC-USD',
 'profile_id': '1d2bb766-5156-4029-ade7-4d4d94a50ce3',
 'side': 'buy',
 'funds': '9.9502487500000000',
 'specified_funds': '10.0000000000000000',
 'type': 'market',
 'post_only': False,
 'created_at': '2021-07-02T12:10:00.556232Z',
 'done_at': '2021-07-02T12:10:00.561Z',
 'done_reason': 'filled',
 'fill_fees': '0.0497502532800000',
 'filled_size': '0.00030121',
 'executed_value': '9.9500506560000000',
 'status': 'done',
 'settled': True}